In [18]:
import sys
import pickle
import logging, os
from time import time
import numpy as np
import pyvista as pv
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import axes3d
%matplotlib notebook

import ipyparallel as ipp

from bem import Electrodes, Sphere, Cylinder, Mesh, Grid, Configuration, Result
from bem.formats import stl
from bem.bemColors_lib.bemColors import bemColors
from helper_functions import helper_functions
from collections import OrderedDict

In [19]:
# prefix = "Calcium_trap_planar_cood_fixed v5"
# path = """C:\Users\Photonics\Documents\GitHub\integrated_photonics_bem\examples\Integrated_photonics\inter_results\htrap\htrap.stl"""
prefix = "htrap"

In [20]:
#radius is the radius of the sphere from the ions location to generate a finer
#electrode mesh
radius= 500e-3
#'area' is the area of the triangles inside the sphere
area = 1e-4
# file = 'mit_charge'
# file = 'htrap_cutout_overhang_taper_2'
# file_out = 'htrap_cutout_overhang_taper_el3'
# stl_path = 'inter_results/taper_overhang/'+file+'.stl'
# fout_name = 'inter_results/taper_overhang/'+file_out+'_'+str(radius)+'_'+str(area)+'.pkl'
mesh_unit = 1e-3

rad = radius*3
inside=area*30
outside=1

zl = -450*1e-3
yl = 75*1e-3
xl = 3.75*1e-3
scale = 1e-8    # rescale factor 1 um.
use_stl = True

# load electrode faces from colored stl
# s_nta is intermediate processed stl file.
s_nta = stl.read_stl(open("%s.stl" % prefix, "rb"))
print("Import stl:",os.path.abspath("./"+prefix+".stl"),"\n")
print("Electrode colors (numbers):\n")

Import stl: c:\Users\Photonics\Documents\GitHub\integrated_photonics_bem\Weiwei_simulation\htrap.stl 

Electrode colors (numbers):



In [21]:
ele_col = bemColors(np.array(list(set(s_nta[2]))),('fusion360','export_stl'))
ele_col.set_my_color(value = (178,178,178),cl_format = ('fusion360','export_stl','RGBA64'),name = 'self_defined')
ele_col.print_stl_colors()

COLORS in the stl:
['bem6']
['bem13']
['bem19']
['bem11']
['bem17']
['bem15']
['bem30']
['bem1']
['bem12']
['bem5']
['bem25']
['bem18']
['bem16']
['bem3']
['bem7']
['bem9']
['bem2']
['bem4']
['bem20']
['bem10']
['bem14']
['bem8']
['bem21']
['_unkColor0']
TOTAL COLORS:  24


In [22]:
# assign a name for each color
# taper htrap_cutout 2022
# assign a name for each color

# In od[key] = value, key is the color from the Fusion360 color library we made
# value assigns a specific name to that color in this program. 
# In future version, we may want to consider getting rid of this step for simplification
# then you just assign the color in fusion 360 and be done with it. The only concern there is you
from collections import OrderedDict
od = OrderedDict()
od['bem1']='DC1'
od['bem2']='DC2'
od['bem3']='DC3'
od['bem4']='DC4'
od['bem5']='DC5'
od['bem6']='DC6'
od['bem7']='DC7'
od['bem8']='DC8'
od['bem9']='DC9'
od['bem10']='DC10'
od['bem11']='DC11'
od['bem12']='DC12'
od['bem13']='DC13'
od['bem14']='DC14'
od['bem15']='DC15'
od['bem16']='DC16'
od['bem17']='DC17'
od['bem18']='DC18'
od['bem19']='DC19'
od['bem20']='DC20'
od['bem21'] = 'DC21'
od['bem25'] = 'RF'
od['bem30'] = 'gnd'
for key in list(od.keys()):
    ele_col.color_electrode(color=key,name=od[key])

# print colors still with no name. These meshes will be neglected in the code below. 
ele_col.drop_colors()

# read stl into mesh with electrode names
# unnamed meshes will not be imported at all

mesh = Mesh.from_mesh(stl.stl_to_mesh(*s_nta, scale=1,
    rename=ele_col.electrode_colors, quiet=True))

dropping color ['_unkColor0']
TOTAL COLORS DROPPED:  1
dropping 20083


In [23]:
mesh.plot()

In [24]:
print(len(mesh.points))

3061


In [25]:
mesh.triangulate(opts="q20Q",new = False);

start triangulate RF
final opts q20Qzr
final opts q20Qzr
final opts q20Qzr
final opts q20Qzr
final opts q20Qzr
final opts q20Qzr
final opts q20Qzr
final opts q20Qzr
final opts q20Qzr
final opts q20Qzr
final opts q20Qzr
final opts q20Qzr
final opts q20Qzr
final opts q20Qzr
final opts q20Qzr
final opts q20Qzr
final opts q20Qzr
finish triangulate RF
start triangulate DC21
final opts q20Qzr
final opts q20Qzr
final opts q20Qzr
final opts q20Qzr
final opts q20Qzr
final opts q20Qzr
final opts q20Qzr
finish triangulate DC21
start triangulate DC2
final opts q20Qzr
final opts q20Qzr
final opts q20Qzr
final opts q20Qzr
final opts q20Qzr
final opts q20Qzr
final opts q20Qzr
final opts q20Qzr
final opts q20Qzr
final opts q20Qzr
final opts q20Qzr
final opts q20Qzr
final opts q20Qzr
finish triangulate DC2
start triangulate DC7
final opts q20Qzr
final opts q20Qzr
final opts q20Qzr
final opts q20Qzr
final opts q20Qzr
final opts q20Qzr
final opts q20Qzr
final opts q20Qzr
final opts q20Qzr
final opts q20Q

In [26]:
mesh.plot()

In [27]:
print(len(mesh.points))

6925


In [28]:
# mesh.areas_from_constraints(Sphere(center=np.array([0.0, 0.0, -9.0]),
#            radius=5, inside=0.01, outside=1.))    # "inside", "outside" set different mesh densities.

mesh.areas_from_constraints(Sphere(center=np.array([xl,yl,zl]),radius=rad, inside=inside, outside=outside))

# mesh.areas_from_constraints(Sphere(center=np.array([0., 0., 1]),
#            radius=3.5, inside=0.1, outside=5))    # "inside", "outside" set different mesh densities.
# retriangulate quality and quiet with areas
mesh.triangulate(opts="q20Q", new=False)
# save base mesh to vtk
mesh.to_vtk(prefix)
print("Output vtk:",os.path.abspath("./"+prefix+".vtk"))    # output path


start triangulate RF
final opts q20Qzra
final opts q20Qzra
final opts q20Qzra
final opts q20Qzra
final opts q20Qzra
final opts q20Qzra
final opts q20Qzra
final opts q20Qzra
final opts q20Qzra
final opts q20Qzra
final opts q20Qzra
final opts q20Qzra
final opts q20Qzra
final opts q20Qzra
final opts q20Qzra
final opts q20Qzra
final opts q20Qzra
finish triangulate RF
start triangulate DC21
final opts q20Qzra
final opts q20Qzra
final opts q20Qzra
final opts q20Qzra
final opts q20Qzra
final opts q20Qzra
final opts q20Qzra
finish triangulate DC21
start triangulate DC2
final opts q20Qzra
final opts q20Qzra
final opts q20Qzra
final opts q20Qzra
final opts q20Qzra
final opts q20Qzra
final opts q20Qzra
final opts q20Qzra
final opts q20Qzra
final opts q20Qzra
final opts q20Qzra
final opts q20Qzra
final opts q20Qzra
finish triangulate DC2
start triangulate DC7
final opts q20Qzra
final opts q20Qzra
final opts q20Qzra
final opts q20Qzra
final opts q20Qzra
final opts q20Qzra
final opts q20Qzra
final o

In [29]:
mesh.plot()

In [30]:
print(len(mesh.points))

10542


In [31]:
fout_name = prefix + "_mesh.pkl"

# save base mesh to a pickle file
with open(fout_name,'wb') as f:
    data = (mesh_unit,
            xl,
            yl,
            zl,
            mesh,
            list(od.values()))
    pickle.dump(data,f)

In [32]:
file_in_name = fout_name

with open(file_in_name,'rb') as f:
    mesh_unit,xl,yl,zl,mesh,electrode_names= pickle.load(f) # import results from mesh processing
# grid to evalute potential and fields atCreate a grid in unit of scaled length mesh_unit. Only choose the interested region (trap center) to save time.
Lx, Ly, Lz = 11*1e-3,11*1e-3,11*1e-3# in the unit of scaled length mesh_unit
# xl,yl,zl = -3.75*1e-3,72*1e-3,270*1.0e-3
xl,yl,zl = 3.75e-3,75e-3,-450e-3
s = 1e-3
sx,sy,sz = s,s,s
print("done")
# ni is number of grid points, si is step size. To  on i direction you need to fix ni*si.
nx, ny, nz = [int(Lx/sx),int(Ly/sy),int(Lz/sz)]
print("Size/l:", Lx, Ly, Lz)
print("Step/l:", sx, sy, sz)
print("Shape (grid point numbers):", nx, ny, nz)
grid = Grid(center=(xl,yl,zl), step=(sx, sy, sz), shape=(nx,ny,nz))
# Grid center (nx, ny ,nz)/2 is shifted to origin
print("lowval",grid.indices_to_coordinates([0,0,0]))
print("Grid center index", grid.indices_to_coordinates((nx/2,ny/2,nz/2)))
print("gridpts:",nx*ny*nz)
center = (xl,yl,zl)
step = (sx,sy,sz)
shape = (nx,ny,nz)
# xyz = np.array([np.linspace(c-s*(h-1)/2., c+s*(h-1)/2.,h)
#                 for c, s, h in zip(center, step, shape)])

done
Size/l: 0.011 0.011 0.011
Step/l: 0.001 0.001 0.001
Shape (grid point numbers): 11 11 11
lowval [-0.00125  0.07    -0.455  ]
Grid center index [ 0.00425  0.0755  -0.4495 ]
gridpts: 1331


In [14]:
def run_job(args):
    # job is Configuration instance.
    job, grid, prefix = args
    # refine twice adaptively with increasing number of triangles, min angle 15 deg.
#     job.adapt_mesh(triangles=4e2, opts="q0.1Q")
#     job.adapt_mesh(triangles=1e3, opts="q0.1Q")
    # solve for surface charges
    job.solve_singularities(num_mom=4, num_lev=3, max_iter=400)
    # get potentials and fields
    # RF_field = (job.name=="RF1") or (job.name=="RF2") or (job.name=="RF0")
    RF_field = "RF" in job.name
    result = job.simulate(grid, field=RF_field, num_lev=2)    # For "RF", field=True computes the field.
    result.to_vtk(prefix)
    print("finished job %s" % job.name)
    return job.collect_charges()

In [15]:
# grid to evalute potential and fields atCreate a grid in unit of scaled length l. Only choose the interested region (trap center) to save time.
s = 0.005 # step 0.01*100 (scale) = 1 um
Lx, Ly, Lz = 0.6, 0.3, 0.2   # in the unit of scaled length l, z=0 is the bottom of substrate, z = 30um is the top of metal
sx, sy, sz = s, s, s
# ni is grid point number, si is step size. Thus to fix size on i direction you need to fix ni*si.
# nx, ny, nz = [2*np.ceil(L/2.0/s).astype('int')+1 for L in (Lx, Ly, Lz)]
nx = 2*np.ceil(Lx/2.0/sx).astype('int')+1 
ny = 2*np.ceil(Ly/2.0/sy).astype('int')+1 
nz = 2*np.ceil(Lz/2.0/sz).astype('int')+1 
print("Size/l:", Lx, Ly, Lz)
print("Step/l:", sx, sy, sz)
print("Shape (grid point numbers):", nx, ny, nz)
grid = Grid(center=(0., 0.0, -9.0), step=(sx, sy, sz), shape=(nx, ny, nz))
# Grid center (nx, ny ,nz)/2 is shifted to origin
print("Grid origin/l:", grid.get_origin()[0])
x, y, z = grid.to_xyz()

Size/l: 0.6 0.3 0.2
Step/l: 0.005 0.005 0.005
Shape (grid point numbers): 121 61 41
Grid origin/l: -0.3


In [16]:
pv.global_theme.notebook = False
def plot_grid_mesh_stl(grid, mesh):
    plotter = pv.Plotter()
    
    # Convert the mesh to PolyData and add it to the plotter
    pd = mesh.to_polydata()
    plotter.add_mesh(pd, scalars='electrode_name', show_edges=True)
    
    # Add the grid points
    x, y, z = np.meshgrid(grid.to_xyz()[0], grid.to_xyz()[1], grid.to_xyz()[2])
    grid_points = np.vstack([x.ravel(), y.ravel(), z.ravel()]).T
    plotter.add_points(grid_points, color='red', point_size=5)
    
    plotter.show_bounds()
    plotter.add_camera_orientation_widget()
    plotter.show()

# Call the function to plot everything together
plot_grid_mesh_stl(grid, mesh)

In [17]:
jobs = list(Configuration.select(mesh, "RF"))    # select() picks one electrode each time.
# jobs = list(Configuration.select(mesh, "RF"))    # select() picks one electrode each time.
# jobs = list(Configuration.select(mesh, "DC.*")) 

# parallel computation
mycluster = ipp.Cluster()
mycluster.start_cluster_sync()
c = mycluster.connect_client_sync()
c.wait_for_engines()

t0 = time()
# Run a parallel map, executing the wrapper function on indices 0,...,n-1
lview = c.load_balanced_view()
# Cause execution on main process to wait while tasks sent to workers finish
lview.block = True 
asyncresult = lview.map_async(run_job, ((job, grid, prefix) for job in jobs))   # Run calculation in parallel
asyncresult.wait_interactive()
print("Computing time: %f s"%(time()-t0))

Starting 32 engines with <class 'ipyparallel.cluster.launcher.LocalEngineSetLauncher'>


KeyboardInterrupt: 

In [18]:
print(len(mesh.points))

9768


In [19]:
import ipyparallel as ipp
from time import time
import psutil

# Select jobs
jobs = list(Configuration.select(mesh, "DC"))

# Parallel computation setup
mycluster = ipp.Cluster()
mycluster.start_cluster_sync()
c = mycluster.connect_client_sync()
c.wait_for_engines()

# Limit engines to 30
selected_engines = c.ids[:30] if len(c.ids) > 30 else c.ids

# Track memory usage function
def check_memory():
    mem = psutil.virtual_memory()
    print(f"Memory Usage: {mem.percent}% - Available: {mem.available / 1e9:.2f} GB")

# Create a direct view on selected engines
lview = c.direct_view(targets=selected_engines)
lview.block = True

# Run calculation in parallel
t0 = time()
check_memory()

# Run in smaller batches to avoid memory overload
batch_size = min(100, len(jobs))  # Adjust batch size as needed
for i in range(0, len(jobs), batch_size):
    asyncresult = lview.map_async(run_job, ((job, grid, prefix) for job in jobs[i:i + batch_size]))
    asyncresult.wait_interactive()
    check_memory()

print("Computing time: %f s" % (time() - t0))


Starting 32 engines with <class 'ipyparallel.cluster.launcher.LocalEngineSetLauncher'>


  0%|          | 0/32 [00:00<?, ?engine/s]

Memory Usage: 37.2% - Available: 43.01 GB


run_job:   0%|          | 0/20 [00:00<?, ?tasks/s]

Memory Usage: 13.8% - Available: 59.00 GB
Computing time: 828.921382 s


In [20]:
jobs = list(Configuration.select(mesh, "DC"))
for job in jobs:
    run_job((job, grid, prefix))

c:\Users\Photonics\.conda\envs\bem39\lib\site-packages\pyvista\core\grid.py:514: PyVistaDeprecationWarning: `dims` argument is deprecated. Please use `dimensions`.
  warnings.warn(


finished job DC14


c:\Users\Photonics\.conda\envs\bem39\lib\site-packages\pyvista\core\grid.py:514: PyVistaDeprecationWarning: `dims` argument is deprecated. Please use `dimensions`.
  warnings.warn(


finished job DC1


c:\Users\Photonics\.conda\envs\bem39\lib\site-packages\pyvista\core\grid.py:514: PyVistaDeprecationWarning: `dims` argument is deprecated. Please use `dimensions`.
  warnings.warn(


...

c:\Users\Photonics\.conda\envs\bem39\lib\site-packages\pyvista\core\grid.py:514: PyVistaDeprecationWarning: `dims` argument is deprecated. Please use `dimensions`.
  warnings.warn(


finished job DC19


c:\Users\Photonics\.conda\envs\bem39\lib\site-packages\pyvista\core\grid.py:514: PyVistaDeprecationWarning: `dims` argument is deprecated. Please use `dimensions`.
  warnings.warn(


finished job DC3
finished job DC16


c:\Users\Photonics\.conda\envs\bem39\lib\site-packages\pyvista\core\grid.py:514: PyVistaDeprecationWarning: `dims` argument is deprecated. Please use `dimensions`.
  warnings.warn(


In [21]:
jobs = list(Configuration.select(mesh, "DC"))    # select() picks one electrode each time.
# jobs = list(Configuration.select(mesh, "RF"))    # select() picks one electrode each time.
# jobs = list(Configuration.select(mesh, "DC.*")) 



# parallel computation
mycluster = ipp.Cluster()
mycluster.start_cluster_sync()
c = mycluster.connect_client_sync()
c.wait_for_engines()

t0 = time()
# Run a parallel map, executing the wrapper function on indices 0,...,n-1
lview = c.load_balanced_view()
# Cause execution on main process to wait while tasks sent to workers finish
lview.block = True 
asyncresult = lview.map_async(run_job, ((job, grid, prefix) for job in jobs))   # Run calculation in parallel
asyncresult.wait_interactive()
print("Computing time: %f s"%(time()-t0))

Starting 32 engines with <class 'ipyparallel.cluster.launcher.LocalEngineSetLauncher'>


  0%|          | 0/32 [00:00<?, ?engine/s]

run_job:   0%|          | 0/20 [00:00<?, ?tasks/s]

Computing time: 811.607075 s


In [22]:
Result.view(prefix, 'DC1')

In [23]:
Result.view(prefix, 'RF')

In [ ]:
jobs = list(Configuration.select(mesh, "GND"))    # select() picks one electrode each time.
# jobs = list(Configuration.select(mesh, "RF"))    # select() picks one electrode each time.
# jobs = list(Configuration.select(mesh, "DC.*")) 

# parallel computation
mycluster = ipp.Cluster()
mycluster.start_cluster_sync()
c = mycluster.connect_client_sync()
c.wait_for_engines()

t0 = time()
# Run a parallel map, executing the wrapper function on indices 0,...,n-1
lview = c.load_balanced_view()
# Cause execution on main process to wait while tasks sent to workers finish
lview.block = True 
asyncresult = lview.map_async(run_job, ((job, grid, prefix) for job in jobs))   # Run calculation in parallel
asyncresult.wait_interactive()
print("Computing time: %f s"%(time()-t0))

Starting 32 engines with <class 'ipyparallel.cluster.launcher.LocalEngineSetLauncher'>


  0%|          | 0/32 [00:00<?, ?engine/s]

run_job:   0%|          | 0/1 [00:00<?, ?tasks/s]

Computing time: 131.516791 s


In [25]:
Result.view(prefix, 'GND')

In [31]:
Result.view(prefix, 'GND')